In [ ]:
#%load_ext autoreload
#%autoreload 2

In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import pca_pseudo_grid_search

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[3]
dataset_name = datasets[1]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Adaptación


## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/twitter/pca/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en una lista, la cual es almacenada en: models/twitter/pca/me1_models_paths.pkl

```python
paths_list = [ruta1, ..., ruta_n]
```

## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = ruta,
    .
    .
    .
    [dominio_n] = ruta,
}
```

Este diccionario queda almacenado en la ruta:
    models/twitter/pca/me1_best_models.pkl

# Pruebas con el dataset Twitter (2000 Dimensiones)

In [2]:
dims = 2000

In [3]:
print tipo
print dataset_name
print dims
print data_path

pca
twitter
2000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [6]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X[: , :dims])

print X.shape

print "Todos los datos disponibles obtenidos"

(4635, 2000)
Todos los datos disponibles obtenidos


In [7]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'n_components': [int(dims/2), int(dims/3), int(dims/4)]
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_%d_" % dims
    
    paths_list = pca_pseudo_grid_search(X, parametros, folder_path, prefix)
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
	n_components: 1000
	Entrenando modelo PCA...
	Guardando modelo en models/twitter/pca/me1_2000_0.pkl

	n_components: 666
	Entrenando modelo PCA...
	Guardando modelo en models/twitter/pca/me1_2000_1.pkl

	n_components: 500
	Entrenando modelo PCA...
	Guardando modelo en models/twitter/pca/me1_2000_2.pkl


Creacion de modelos terminada
Guardando rutas en models/twitter/pca/me1_2000_models_paths.pkl
Rutas cargadas en la variable 'paths_list'


In [8]:
print "Modelos almacenados en:"
for ruta in paths_list:
    print "\t", ruta

Modelos almacenados en:
	models/twitter/pca/me1_2000_0.pkl
	models/twitter/pca/me1_2000_1.pkl
	models/twitter/pca/me1_2000_2.pkl


In [9]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for pca_model_path in paths_list:
            
            pca_model = joblib.load(pca_model_path)
            n_components = pca_model.n_components_
            
            print "t%d) n_components=%d " % (i, n_components),
            # se obtienen los datos del dominio
            X_tr = labeled[domain]['X_tr'][:, :dims].todense()
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = pca_model.transform(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = pca_model_path
                best_model = pca_model
            i = i+1
        
        #se guarda el mejor modelo para este dominio
        print "tMejor modelo: n_components=%d" % (best_model.n_components_)
        best_models[domain] = best_path
        
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
rio2016
t1) n_components=1000  score: 0.8854
t2) n_components=666  score: 0.8878
t3) n_components=500  score: 0.8760
tMejor modelo: n_components=666
thevoice
t1) n_components=1000  score: 0.9063
t2) n_components=666  score: 0.9024
t3) n_components=500  score: 0.8938
tMejor modelo: n_components=1000
general
t1) n_components=1000 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 score: 0.7133
t2) n_components=666 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 score: 0.7171
t3) n_components=500 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


 score: 0.7081
tMejor modelo: n_components=666
Rutas guardadas en  models/twitter/pca/me1_2000_best_models.pkl


In [10]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

thevoice - models/twitter/pca/me1_2000_0.pkl
rio2016 - models/twitter/pca/me1_2000_1.pkl
general - models/twitter/pca/me1_2000_1.pkl


In [11]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        pca_model = joblib.load(best_model_path)
        
        print "\tAdaptando dominio"
        X_tr = labeled[domain]['X_tr'][:, :dims].todense()

        tr_reps = pca_model.transform(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para rio2016
	Adaptando dominio
Cargando mejor modelo para thevoice
	Adaptando dominio
Cargando mejor modelo para general
	Adaptando dominio
3/3 dominios adaptados


## Pruebas de clasificación

In [12]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    modelo = joblib.load(ruta)
    
    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)


    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### PCA ####
    #############
    #entrenado en dominio src y probado en dominio tgt adaptados
    X_tr_a = adapted[src]['X_tr']
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts_a = modelo.transform(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    print "Entrenando clasificador adaptado."
    svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-svc_a.score(X_ts_a, y_ts)

    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i+=1

print "\nPruebas completadas"

Tarea 1 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 3 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 4 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 5 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 6 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.

Pruebas completadas


In [13]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,PCA,r->t,rio2016,thevoice,7.362689,21.875000,14.512311
1,PCA,r->g,rio2016,general,26.998127,43.143803,16.145676
2,PCA,t->r,thevoice,rio2016,10.208711,14.745917,4.537205
3,PCA,t->g,thevoice,general,26.998127,40.397009,13.398882
4,PCA,g->r,general,rio2016,10.208711,22.323049,12.114338
5,PCA,g->t,general,thevoice,7.362689,18.750000,11.387311


In [14]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/pca/me1_2000.csv
Resultados guardados.


# Pruebas con el dataset Twitter (1000 Dimensiones)

In [15]:
dims = 1000

In [16]:
print tipo
print dataset_name
print dims
print data_path

pca
twitter
1000
data


In [17]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [18]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X[: , :dims])

print X.shape

print "Todos los datos disponibles obtenidos"

(4635, 1000)
Todos los datos disponibles obtenidos


In [19]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'n_components': [int(dims/2), int(dims/3), int(dims/4)]
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_%d_" % dims
    
    paths_list = pca_pseudo_grid_search(X, parametros, folder_path, prefix)
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
	n_components: 500
	Entrenando modelo PCA...
	Guardando modelo en models/twitter/pca/me1_1000_0.pkl

	n_components: 333
	Entrenando modelo PCA...
	Guardando modelo en models/twitter/pca/me1_1000_1.pkl

	n_components: 250
	Entrenando modelo PCA...
	Guardando modelo en models/twitter/pca/me1_1000_2.pkl


Creacion de modelos terminada
Guardando rutas en models/twitter/pca/me1_1000_models_paths.pkl
Rutas cargadas en la variable 'paths_list'


In [20]:
print "Modelos almacenados en:"
for ruta in paths_list:
    print "\t", ruta

Modelos almacenados en:
	models/twitter/pca/me1_1000_0.pkl
	models/twitter/pca/me1_1000_1.pkl
	models/twitter/pca/me1_1000_2.pkl


In [21]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for pca_model_path in paths_list:
            
            pca_model = joblib.load(pca_model_path)
            n_components = pca_model.n_components_
            
            print "t%d) n_components=%d " % (i, n_components),
            # se obtienen los datos del dominio
            X_tr = labeled[domain]['X_tr'][:, :dims].todense()
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = pca_model.transform(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = pca_model_path
                best_model = pca_model
            i = i+1
        
        #se guarda el mejor modelo para este dominio
        print "tMejor modelo: n_components=%d" % (best_model.n_components_)
        best_models[domain] = best_path
        
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
rio2016
t1) n_components=500  score: 0.8794
t2) n_components=333  score: 0.8765
t3) n_components=250  score: 0.8617
tMejor modelo: n_components=500
thevoice
t1) n_components=500  score: 0.8451
t2) n_components=333  score: 0.8462
t3) n_components=250  score: 0.8363
tMejor modelo: n_components=333
general
t1) n_components=500  score: 0.6596
t2) n_components=333  score: 0.6570
t3) n_components=250  score: 0.6508
tMejor modelo: n_components=500
Rutas guardadas en  models/twitter/pca/me1_1000_best_models.pkl


In [22]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

thevoice - models/twitter/pca/me1_1000_1.pkl
rio2016 - models/twitter/pca/me1_1000_0.pkl
general - models/twitter/pca/me1_1000_0.pkl


In [23]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        pca_model = joblib.load(best_model_path)
        
        print "\tAdaptando dominio"
        X_tr = labeled[domain]['X_tr'][:, :dims].todense()

        tr_reps = pca_model.transform(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para rio2016
	Adaptando dominio
Cargando mejor modelo para thevoice
	Adaptando dominio
Cargando mejor modelo para general
	Adaptando dominio
3/3 dominios adaptados


## Pruebas de clasificación

In [24]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    modelo = joblib.load(ruta)
    
    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)


    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### PCA ####
    #############
    #entrenado en dominio src y probado en dominio tgt adaptados
    X_tr_a = adapted[src]['X_tr']
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts_a = modelo.transform(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    print "Entrenando clasificador adaptado."
    svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-svc_a.score(X_ts_a, y_ts)

    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i+=1

print "\nPruebas completadas"

Tarea 1 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 3 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 4 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 5 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 6 de 6
Cargando modelo existente.
Entrenando clasificador adaptado.

Pruebas completadas


In [25]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,PCA,r->t,rio2016,thevoice,19.045928,25.260417,6.214489
1,PCA,r->g,rio2016,general,30.407980,44.284475,13.876495
2,PCA,t->r,thevoice,rio2016,7.372958,16.084392,8.711434
3,PCA,t->g,thevoice,general,30.407980,41.571190,11.163210
4,PCA,g->r,general,rio2016,7.372958,17.127949,9.754991
5,PCA,g->t,general,thevoice,19.045928,21.969697,2.923769


In [26]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/pca/me1_1000.csv
Resultados guardados.
